# Importanto bibliotecas para o trabalho.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

## Adicionando dados para manipulação

A base de dados que vamos utilizar estara disponivel em: [link para base de dados](https://www.kaggle.com/datasets/spscientist/students-performance-in-exams/data)

In [ ]:
df = pd.read_csv('dados/performaceEstudantes.csv')

Mostrando informações basicas do frame.

In [ ]:
df.info() # utilizado para ver as informações do dataframe.
print("\n\n")
df['gender'].value_counts() # conta de forma agrupada

# Analise dos dados

vamos fazer a media das notas da provas de matematica, leitura e escrita.
Logo em seguida vamos remover as tabelas das notas separadas, para futuramente usar para a rede neural.

In [ ]:
df['media'] = df[['math score', 'reading score', 'writing score']].mean(axis=1)

In [ ]:
df.drop('reading score', axis=1, inplace=True)
df.drop('writing score', axis=1, inplace=True)
df.drop('math score', axis=1, inplace=True)

df.info()

Agora que o banco de dados esta de acordo vamos fazer a moda, media, mediana, variação, desvio padrao, minimo e maximo.

In [ ]:
# A COLUNA QUE VAMOS UTILIZAR É MEDIA
column = 'media'

moda = df[column].mode().values[0]  # Retorna um array, então pegamos o primeiro valor # Calcular a moda

media = df[column].mean()# Calcular a média

mediana = df[column].median()# Calcular a mediana

variacao = df[column].var()# Calcular a variação

desvio_padrao = df[column].std()# Calcular o desvio padrão

minimo = df[column].min()# Calcular o mínimo

maximo = df[column].max()# Calcular o máximo

print(f'Moda: {moda}\n')
print(f'Média: {media}\n')
print(f'Mediana: {mediana}\n')
print(f'Variação: {variacao}\n')
print(f'Desvio Padrão: {desvio_padrao}\n')
print(f'Mínimo: {minimo}\n')
print(f'Máximo: {maximo}\n')


### aqui vamos fazer a distribuição das notas de pessoas

In [ ]:

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(df['media'], bins=15, kde=True, color='blue', edgecolor='black')

# Adicionar títulos e rótulos
plt.title('Distribuição das Notas dos Alunos')
plt.xlabel('Nota')
plt.ylabel('Número de Pessoas')

# Mostrar o gráfico
plt.show()

### Media da nota pelo nivel de graduação dos pais.

In [ ]:

mean_scores = df.groupby('parental level of education')['media'].mean().sort_values()

# Criar o gráfico de caixa (BARPLOT)
plt.figure(figsize=(10, 6))
sns.barplot(x=mean_scores.index, y=mean_scores.values, palette='viridis')

# Adicionar títulos e rótulos
plt.title('Média das Notas por Nível de Escolaridade dos pais')
plt.xlabel('Nível de Escolaridade')
plt.ylabel('Media das notas')

# Exibir o gráfico
plt.xticks(rotation=45)  # Rotacionar rótulos do eixo x para ver melhor
plt.show()

### Media das notas por grupo etnico

In [ ]:
mean_scores = df.groupby('race/ethnicity')['media'].mean().sort_values()

# Converter para DataFrame
mean_scores_df = mean_scores.reset_index()
mean_scores_df.columns = ['race/ethnicity', 'media']

# Criar o gráfico de barras (Barplot)
plt.figure(figsize=(10, 6))
sns.barplot(data=mean_scores_df, x='race/ethnicity', y='media', palette='viridis')

# Adicionar títulos e rótulos
plt.title('Média das notas por etnia')
plt.xlabel('Grupos')
plt.ylabel('Média das notas')

# Exibir o gráfico
plt.xticks(rotation=45)  # Rotacionar rótulos do eixo x para melhor legibilidade
plt.show()

# Tratamento de dados para a rede Neural: 

Aqui, vamos converter os dados alfabéticos em numéricos.

In [ ]:
df['gender'] = df['gender'].replace({'female': 0, 'male': 1})

df['race/ethnicity'] = df['race/ethnicity'].replace({ 'group A' : 1,'group B' : 2,'group C': 3,'group D' : 4,'group E' : 5})

df['parental level of education'] = df['parental level of education'].replace({'high school' : 1, 'some high school' : 2, 'some college' : 3 , "associate's degree" : 4, "bachelor's degree": 5, "master's degree" : 6})

df['lunch'] = df['lunch'].replace({'standard': 1, 'free/reduced':0})
df['test preparation course'] = df['test preparation course'].replace({'none': 0, 'completed':1})

### Agora vamos fazer a construção e treinamento da rede neural.

In [ ]:
###   df.info()
X = df.drop('media', axis=1) #coluna que sera o nosso ponto de chegada
y = df['media']

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Construção do modelo
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Sem função de ativação na camada de saída para regressão
])

# Compilação do modelo com função de perda apropriada
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

# Treinamento do modelo
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2)

# Avaliação do modelo
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error: {mae}')

## Exemplo e utilização

In [ ]:
# Vamos supor que a pessoa seja: 
# Male: 1 | Group B: 2 | bachelor's degree: 4 | free/reduced: 0 | completed: 1
# Observando sempre a ordem correta dos dados

# Usaremos as colunas(que contem o nume de cada propriedade) de X pois ela ja esta sem a media
dados = {
    'gender': [1],
    'race/ethnicity': [2],
    'parental level of education': [4],
    'lunch': [0],
    'test preparation course': [1]
}

# Novo dado de entrada como DataFrame
novo_dado_df = pd.DataFrame(dados)

# Normalizar o novo dado usando o mesmo scaler que foi ajustado nos dados de treinamento
nnovo_dado_df = pd.DataFrame(dados, columns=X.columns)

# Fazer a previsão usando o modelo treinado
previsao = model.predict(novo_dado_normalizado)

print(f'Previsão da média: {previsao[0][0]}\nCom uma taxa de erro de 10 pontos para cima ou para baixo')